In [75]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

In [76]:
df = pd.read_csv('diabetes.csv') 
print(df.shape)
df.describe().transpose()

(768, 9)


,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [77]:
target_column = ['Outcome'] 
object = StandardScaler()
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = object.fit_transform(df[predictors])
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,-6.476301e-17,1.000652,-1.141852,-0.844885,-0.250952,0.639947,3.906578
Glucose,768.0,-9.251859e-18,1.000652,-3.783654,-0.685236,-0.121888,0.605771,2.444478
BloodPressure,768.0,1.503427e-17,1.000652,-3.572597,-0.367337,0.149641,0.563223,2.734528
SkinThickness,768.0,1.006140e-16,1.000652,-1.288212,-1.288212,0.154533,0.719086,4.921866
Insulin,768.0,-3.006854e-17,1.000652,-0.692891,-0.692891,-0.428062,0.412008,6.652839
BMI,768.0,2.590520e-16,1.000652,-4.060474,-0.595578,0.000942,0.584771,4.455807
DiabetesPedigreeFunction,768.0,2.451743e-16,1.000652,-1.189553,-0.688969,-0.300128,0.466227,5.883565
Age,768.0,1.931325e-16,1.000652,-1.041549,-0.786286,-0.360847,0.660206,4.063716
Outcome,768.0,3.489583e-01,0.476951,0.000000,0.000000,0.000000,1.000000,1.000000


In [78]:
X = df[predictors].values
y = df[target_column].values
test_size=0.30
develop_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=40)
X_trainnodev = X_train
X_testnodev = X_test
y_trainnodev = y_train
y_testnodev = y_test
newX_train = X_train
newX_test = X_test
newy_train = y_train
newy_test = y_test
post_split_develop_size = develop_size / (1 - test_size)
X_train, X_develop, y_train, y_develop = train_test_split(X_train, y_train, test_size=post_split_develop_size, random_state=40)
print(X_train.shape); print(X_test.shape); print(X_develop.shape)

(383, 8)
(231, 8)
(154, 8)


In [79]:
(8 + 1)/2

4.5

In [80]:
lrc = LogisticRegression(random_state=0, max_iter=1000 ,verbose = True)
lrc.fit(newX_train, newy_train.ravel())
regpred_train = lrc.predict(newX_train)
regpred_test = lrc.predict(newX_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [81]:
print("MATRIZ DE CONFUSAO DO TESTE DA REGRESSAO LOGISTICA");print(confusion_matrix(newy_test,regpred_test))
print(classification_report(newy_test,regpred_test))

MATRIZ DE CONFUSAO DO TESTE DA REGRESSAO LOGISTICA
[[124  18]
 [ 41  48]]
              precision    recall  f1-score   support

           0       0.75      0.87      0.81       142
           1       0.73      0.54      0.62        89

    accuracy                           0.74       231
   macro avg       0.74      0.71      0.71       231
weighted avg       0.74      0.74      0.74       231



In [82]:
mlp = MLPClassifier( max_iter=2000)
mlpdev = MLPClassifier( max_iter=4000)

In [83]:
parameter_space = {
    'hidden_layer_sizes': [(5,5,5),(4,4,4), (8,8,8)],
    'activation': ['tanh', 'relu', 'logistic'],
    'solver': ['sgd', 'adam'],
    'learning_rate': ['constant','adaptive']
    }
random_search = RandomizedSearchCV(mlp, parameter_space,n_iter = 12, cv = 5)
random_search.fit(X_trainnodev,y_trainnodev.ravel())
params = random_search.best_params_
score = random_search.best_score_
print(params);print(score)

{'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (8, 8, 8), 'activation': 'tanh'}
0.7709068881966078


In [84]:
devrandom_search = RandomizedSearchCV(mlpdev, parameter_space,n_iter = 12, cv = 5)
devrandom_search.fit(X_develop,y_develop.ravel())
paramsdev = devrandom_search.best_params_
scoredev = devrandom_search.best_score_
print(paramsdev);print(scoredev)

{'solver': 'sgd', 'learning_rate': 'constant', 'hidden_layer_sizes': (8, 8, 8), 'activation': 'tanh'}
0.7015053763440859


In [85]:
print("test_score regression", accuracy_score(newy_test,regpred_test))

test_score regression 0.7445887445887446


In [86]:
predict_test = random_search.predict(X_testnodev)
testscore = accuracy_score(y_testnodev,predict_test)
print("test_score normal model", testscore)

test_score normal model 0.7575757575757576


In [87]:
predict_testdev = devrandom_search.predict(X_testnodev)
testscore = accuracy_score(y_test,predict_testdev)
print("test_score dev model", testscore)

test_score dev model 0.6623376623376623


In [88]:
print("MATRIZ DE CONFUSAO DO TESTE DO MODELO SEM DESENVOLVIMENTO");print(confusion_matrix(y_test,predict_test))
print(classification_report(y_testnodev,predict_test))

MATRIZ DE CONFUSAO DO TESTE DO MODELO SEM DESENVOLVIMENTO
[[119  23]
 [ 33  56]]
              precision    recall  f1-score   support

           0       0.78      0.84      0.81       142
           1       0.71      0.63      0.67        89

    accuracy                           0.76       231
   macro avg       0.75      0.73      0.74       231
weighted avg       0.75      0.76      0.75       231



In [89]:
print("MATRIZ DE CONFUSAO DO TESTE DO MODELO COM DESENVOLVIMENTO");print(confusion_matrix(y_test,predict_testdev))
print(classification_report(y_test,predict_testdev,zero_division=1))

MATRIZ DE CONFUSAO DO TESTE DO MODELO COM DESENVOLVIMENTO
[[138   4]
 [ 74  15]]
              precision    recall  f1-score   support

           0       0.65      0.97      0.78       142
           1       0.79      0.17      0.28        89

    accuracy                           0.66       231
   macro avg       0.72      0.57      0.53       231
weighted avg       0.70      0.66      0.59       231

